In [1]:
import cv2
import mediapipe as mp
import os
import numpy as np

# Initialize MediaPipe Face Detection and Face Mesh
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
face_detection = mp_face_detection.FaceDetection()
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

# Ensure the output directory exists
output_dir = './face/user_2/'
os.makedirs(output_dir, exist_ok=True)

# Initialize OpenCV video capture
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    # Convert the image to RGB format
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the image for face detection using MediaPipe
    results = face_detection.process(image_rgb)

    if results.detections:
        for detection in results.detections:
            # Get the face landmarks
            face_landmarks = face_mesh.process(image_rgb)

            if face_landmarks.multi_face_landmarks:
                for face_lms in face_landmarks.multi_face_landmarks:
                    count += 1
                    face_roi = image_rgb

                    # Save the face image
                    file_name_path = os.path.join(output_dir, f'{count}.jpg')
                    cv2.imwrite(file_name_path, cv2.cvtColor(face_roi, cv2.COLOR_RGB2BGR))

                    # Draw the face landmarks on the image
                    mp_drawing = mp.solutions.drawing_utils
                    mp_drawing.draw_landmarks(
                        image=frame,
                        landmark_list=face_lms,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1)
                    )

                    # Put count on images and display live count
                    cv2.putText(frame, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                    cv2.imshow('Face Cropper', frame)
            else:
                print("Face Not Found!!")

    if cv2.waitKey(1) == 13 or count == 100:  # Press 'Enter' to exit or collect 100 samples
        break

cap.release()
cv2.destroyAllWindows()
print("Collecting samples complete")

# Train the face recognition model using OpenCV
data_path = './face/user_2/'
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

Training_Data, Labels = [], []

for i, file in enumerate(os.listdir(data_path)):
    image_path = os.path.join(data_path, file)
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if images is None:
        continue
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

Labels = np.asarray(Labels, dtype=np.int32)

if len(Training_Data) == 0:
    print("No training data found. Please ensure there are images in the specified directory.")
    exit()

face_recognizer.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained successfully")

C:\Users\Admin\AppData\Roaming\Python\Python310\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Model trained successfully


In [4]:
# Save the trained model to a .yml file
model_file = './face_recognition_model.yml'
face_recognizer.save(model_file)
print(f"Trained model saved to {model_file}")


Trained model saved to ./face_recognition_model.yml


In [6]:
import os
# Train the face recognition model using OpenCV
data_path = './face/user_2/'
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

Training_Data, Labels = [], []

for i, file in enumerate(os.listdir(data_path)):
    image_path = os.path.join(data_path, file)
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if images is None:
        continue
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

Labels = np.asarray(Labels, dtype=np.int32)

if len(Training_Data) == 0:
    print("No training data found. Please ensure there are images in the specified directory.")
    exit()

# Train the model
face_recognizer.train(np.asarray(Training_Data), np.asarray(Labels))

# Save the trained model to a .yml file
model_file = './face_recognition_model.yml'
face_recognizer.save(model_file)
print(f"Trained model saved to {model_file}")


Trained model saved to ./face_recognition_model.yml


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection

# Create a FaceDetection object
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)



def detect_faces(image):
    # Convert the image to RGB format
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Process the image for face detection
    results = face_detection.process(image_rgb)
    
    # Draw rectangles around the detected faces and print confidence
    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            x, y, w, h = int(bbox.xmin * image.shape[1]), int(bbox.ymin * image.shape[0]), int(bbox.width * image.shape[1]), int(bbox.height * image.shape[0])
            
            # Extract the face ROI
            face_roi = image[y:y+h, x:x+w]
            face_gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
            
            # Perform face recognition
            label, confidence = face_recognizer.predict(face_gray)
            
            # Define bounding box color based on recognition confidence
            if confidence < 50:
                color = (0, 0, 255)  # Red color for low confidence
            else:
                color = (0, 255, 0)  # Green color for high confidence
            
            # Draw the bounding box with the appropriate color
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            
            # Print the confidence
            cv2.putText(image, f"Confidence: {confidence}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    
    return image

# For webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if ret:
        result = detect_faces(frame)
        cv2.imshow('Face Detection', result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
